# Decision trees and Random forest

In this notebook, we explain what a random forest is and when to use it. We then write a simple code using some data set on sklearn to see how it performs.

First, let’s introduce the problem.

We are given a dataset represented by a matrix $X \in \mathbb{R}^{N \times d}$, where:
- $N$ is the number of samples (or data points),
- $d$ is the number of features (or input variables).

Each sample has an associated target value, $y_i$, which are collected in a target vector $y$.

We then **split the dataset** into two subsets:
- a **training set**, used to fit a model (e.g., linear regression or random forest), and
- a **testing set**, used to evaluate the model’s performance.

The model is trained on the training set and then assessed on the test set by comparing its predictions to the actual values in $y$. This helps us estimate how well the model is likely to perform on unseen data.


We have two types of problems

We typically deal with two types of problems:

- **Classification**: In this case, the target vector $y$ takes discrete values from a finite set of size $K$, often represented as $\{0, 1, \dots, K-1\}$.
  
- **Regression**: Here, $y$ takes continuous values in $\mathbb{R}$.

## Decision Tree

We first discuss the **decision tree**, and then its generalization: the **random forest**.

Let:
- $X_{\text{train}}, X_{\text{test}}$ be the two partitions of the input data $X$, where the first is used for training and the second for testing;
- $y_{\text{train}}, y_{\text{test}}$ be the corresponding partitions of the labels $y$.

---

The goal is to find the best way to split $X_{\text{train}}$ into smaller subsets that are increasingly **pure** — meaning they contain samples of only one class. We do this recursively.

We consider **midpoint splitting**. For a fixed feature index $j$, we sort all the values $X^{(i, j)}_{\text{train}}$ across samples $i$. This gives a sorted list of values for that feature.

We then compute a splitting point $t_{ij}$ at each midpoint:

$$
t_{ij} = \frac{1}{2}\left(X^{(i,j)}_{\text{train}} + X^{(i+1,j)}_{\text{train}}\right)
$$

For each candidate threshold $t_{ij}$, we compute the **Gini impurity** (for classification):

$$
G(i,j) = \frac{n_L(i,j)}{n(i,j)} G_L(i,j) + \frac{n_R(i,j)}{n(i,j)} G_R(i,j)
$$

where:
- $G_L$, $G_R$ are the Gini impurities of the left and right partitions;
- $n(i,j)$ is the total number of samples;
- $n_L(i,j)$, $n_R(i,j)$ are the number of samples going to the left and right of threshold $t_{ij}$, respectively.

The Gini impurity for a set is given by:

$$
G = 1 - \sum_{k=1}^K p_k^2
$$

where $p_k$ is the proportion of samples in class $k$, and $K$ is the total number of classes.

We search over all $(t_{ij},j)$ pairs and choose the pair $(t^*, j^*)$ that minimizes $G(i,j)$. This determines the **first split**.

We now have a bipartition of the data based on whether a sample's feature $j^*$ is greater or less than $t^*$. These subsets are called the **left child** and **right child**.

The splitting process is repeated **recursively** on each child node, stopping when:
- The node is **pure** (all samples have the same label, so $G = 0$);
- The **maximum tree depth** is exceeded;
- The node contains **too few samples** (below a predefined threshold).


For **regression**, we use the **mean squared error (MSE)** as the splitting criterion rather than the **Gini impurity** used in classification problems.

The **MSE** is defined as:

$$
MSE = \frac{1}{n} \sum_{i=1}^{n}(y_i - \hat{y})^2
$$

where:
- $y_i$ is the true target value for the $i$-th sample.
- $\hat{y}$ is the predicted target value for the node (usually the **mean** target value of the samples in that node).
- $n$ is the number of samples in the node.

After computing the MSE for each possible split, we choose the split $t^*$ that minimizes the MSE or maximizes the variance reduction. We then proceed with the splitting procedure, recursively applying the same process at each node.

When a node cannot be split any further (due to stopping criteria such as maximum tree depth, minimum number of samples per node, or if the node is pure), we assign $\hat{y}$ to that node, which is the mean of the target values of the samples in the node.

Thus, the final prediction for a new sample is obtained by traversing the tree, starting at the root node and following the splits until reaching a leaf node. The predicted value for the sample is then the value $\hat{y}$ assigned to the leaf node.

### Key Points:
1. **Splitting Criterion**: For regression trees, we use **MSE** or **SSE** to choose the best splits.
2. **Prediction**: The predicted value for a new sample is the mean target value $\hat{y}$ of the samples in the leaf node where the sample falls.


---

For example, a decision tree for classifying flowers into 3 species might look like:

       (petal length ≤ 2.45?)
              /         \
          Yes             No
     (leaf: class 0)    (petal width ≤ 1.75?)
                          /       \
                      Yes         No
                (leaf: class 1) (leaf: class 2)



Once the tree is trained, we classify a new sample $x$ by following the decision rules down the tree until we reach a **leaf node**, where a class label is assigned.

## Random Forest

Random Forest is an **ensemble learning method** that builds on decision trees by combining the predictions of **multiple trees** to improve accuracy and reduce overfitting.

### Main Concept

Instead of using a single decision tree:

- For **classification**: predictions are made by **majority vote** across all trees.
- For **regression**: predictions are made by taking the **average** across all trees.

This reduces the tendency of a single tree to overfit the data.

---

###  How the Forest is trained

#### 1. **Bootstrapping (Bagging)**

- For each tree, we generate a **random sample _with replacement_** from the original dataset.
- This process is called **bootstrapping**.
- We repeat this for $n_{trees}$, training one tree on each sample. We have a collection of $n_{trees}$ number of trees: a forest.

#### 2. **Feature Bagging**

- At each **split** in a tree, we choose the best split from a **random subset of features** rather than all features.
- This further introduces randomness and reduces correlation between trees.

---


# Codes
We now use sklearn to write two codes:
* Classification code: we check the accuracy of random forest vs a single decision tree
* Regression code: we check the accuracy of random forest vs a linear regression

# Classification code

In [18]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

data = load_wine()
X = data.data
y = data.target

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 1. Train a single decision tree
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
tree_accuracy = accuracy_score(y_test, y_pred_tree)

# 2. Train a random forest
forest_clf = RandomForestClassifier(random_state=42)
forest_clf.fit(X_train, y_train)
y_pred_forest = forest_clf.predict(X_test)
forest_accuracy = accuracy_score(y_test, y_pred_forest)

# Accuracy
print(f"Accuracy of Decision Tree: {tree_accuracy:.2f}")
print(f"Accuracy of Random Forest: {forest_accuracy:.2f}")

Accuracy of Decision Tree: 0.94
Accuracy of Random Forest: 1.00


We can see that the random forest has a better accuracy than a single decision tree

# Regression code

In [17]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

data = fetch_california_housing()
X = data.data
y = data.target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Compare Mean Squared Errors
mse_lr = mean_squared_error(y_test, y_pred_lr)
mse_rf = mean_squared_error(y_test, y_pred_rf)

print(f"Linear Regression MSE: {mse_lr:.4f}")
print(f"Random Forest MSE: {mse_rf:.4f}")

Linear Regression MSE: 0.5559
Random Forest MSE: 0.2554


## Linear Regression vs Random Forest

The random forest typically has a lower **mean squared error (MSE)** than linear regression. However, there are some drawbacks.

On one hand, in linear regression we have a clear model:

$$ \hat{y} = X\beta \implies \hat{y}_i = \sum_{j=1}^{d} X_{ij} \beta_j + b $$

Here, $X$ is the $N \times (d+1)$ **design matrix**, where $N$ is the number of samples and $d$ is the number of features. Each row of $X$ encodes the features of a sample, and there is an additional column of ones that corresponds to the constant intercept $b$ of the model.

This implies that we have a clear interpretation of our results: the best-fitting parameter $\beta$ minimizes the MSE, assuming the model is linear.

On the other hand, the random forest **incorporates non-linear features**, but we do not have an explicit expression for the model $\hat{y}$ in terms of $X$. This makes it a **non-parametric** method and makes interpretation harder.

While random forests often perform better in terms of predictive accuracy, especially in capturing complex patterns, they lack the interpretability that linear regression provides.
